# Reinforcement

In [ ]:
# Import data structures folder
import sys
import os

# Go to root software_engineering
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

R-10.1 Give a concrete implementation of the pop method in the context of the MutableMapping class, relying only on the five primary abstract methods of that class.

In [ ]:
from collections import MutableMapping

class PopMutableMap(MutableMapping):

    def pop(self, key, default=None):
        try:
            value = self[key]
            del self[key]
            return value
        except KeyError:
            if default is not None:
                return default
            raise KeyError('key not found')

R-10.2 Give a concrete implementation of the items() method in the context of the MutableMapping class, relying only on the five primary abstract methods of that class. What would its running time be if directly applied to the UnsortedTableMap subclass?

In [ ]:
from collections import MutableMapping

class ItemsMutableMap(MutableMapping):

    def items(self):
        for k in iter(self):
            yield (k, self[k])

"""
If applied to UnsortedTableMap the running time would be O(n^2) because each iteration of the items() loop would do a search of the key in the internal array of UnsortedTableMap O(n * n) = O(n^2)
"""

R-10.3 Give a concrete implementation of the items() method directly within the UnsortedTableMap class, ensuring that the entire iteration runs in O(n) time.

In [ ]:
from data_structures.maps.unsorted_table_map import UnsortedTableMap

class ItemsUnsortedTableMap(UnsortedTableMap):

    def items(self):
        for item in self._table:
            yield (item._key, item._value)

R-10.4 What is the worst-case running time for inserting n key-value pairs into an initially empty map M that is implemented with the UnsortedTableMap class?

The worst-case running time for inserting `n` key-value pairs into an initially empty map M that is implemented with the UnsortedTableMap class is when all the items added have distinct keys, which would result in a O(n^2) worst-case running time because before each insertion a O(n) search is done in the internal array to try to find an existing key.

R-10.5 Reimplement the UnsortedTableMap class from Section 10.1.5, using the PositionalList class from Section 7.4 rather than a Python list.

In [ ]:
from data_structures.maps.map_base import MapBase
from data_structures.linked_lists.positional_list import PositionalList


class UnsortedTableMap(MapBase):
    """Map implementation using an unordered list"""

    def __init__(self):
        self._table = PositionalList()

    def __getitem__(self, k):
        """Return value associated wiht key k (raise KeyError if not found)"""
        for element in self._table:
            if k == element._key:
                return element._value
        raise KeyError(f"Key Error: '{repr(k)}'")
    
    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present"""
        for element in self._table:
            if k == element._key:
                element._value = v
                return
        # did not find match for key
        self._table.add_last(self._Item(k,v))

    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found)"""
        cursor = self._table.first()
        while cursor is not None:
            if cursor.element()._key == k:
                self._table.delete(cursor)
                return
            else:
                cursor = self.after(cursor)
        raise KeyError(f"Key Error: '{repr(k)}'")

    def __len__(self):
        """Return number of items in the map"""
        return len(self._table)
    
    def __iter__(self):
        """Generate iteration of the map's keys"""
        for element in self._table:
            yield element._key


R-10.6 Which of the hash table collision-handling schemes could tolerate a load factor above 1 and which could not?

The covered collision-handling schemes are:

- Separate Chaining (can tolerate a load factor above 1)
- Open Addressing (linear probing, quadratic probing, or double hashing) (cannot tolerate a load factor above 1)

R-10.7 Our Position classes for lists and trees support the `__eq__` method so that two distinct position instances are considered equivalent if they refer to the same underlying node in a structure. For positions to be allowed as keys in a hash table, there must be a definition for the hash method that is consistent with this notion of equivalence. Provide such a `__hash__` method.

In [ ]:
def __hash__(self):
    return hash(self._node)

"""
Since we don't know what the `_node` object is going to be, these objects should be implementing their own definition of the hash method.
"""

R-10.8 What would be a good hash code for a vehicle identification number that is a string of numbers and letters of the form “9X9XX99X9XX999999,” where a “9” represents a digit and an “X” represents a letter?

Polynomical hash codes or cyclic shift hash codes

R-10.9 Draw the 11-entry hash table that results from using the hash function, h(i) = (3i + 5) mod 11, to hash the keys 12, 44, 13, 88, 23, 94, 11, 39, 20, 16, and 5, assuming collisions are handled by chaining.

0 - 13
1 - 94, 39
2 - 
3 - 
4 - 
5 - 44, 88, 11
6 - 
7 - 
8 - 12, 23
9 - 16, 5
10 - 20

R-10.10 What is the result of the previous exercise, assuming collisions are handled by linear probing?

0 - 13
1 - 94
2 - 39
3 - 16
4 - 5
5 - 44
6 - 88
7 - 11
8 - 12
9 - 23
10 - 20

R-10.11 Show the result of Exercise R-10.9, assuming collisions are handled by quadratic probing, up to the point where the method fails.

0 - 13
1 - 94
2 - 39
3 - 11
4 - 
5 - 44
6 - 88
7 - 16
8 - 12
9 - 23
10 - 20

R-10.12 What is the result of Exercise R-10.9 when collisions are handled by double hashing using the secondary hash function h′(k) = 7− (k mod 7)?

In [ ]:
"""
Separate chaining
resolving collisions with double hashing
"""

N = 11
l = [12, 44, 13, 88, 23, 94, 11, 39, 20, 16, 5]
keys = []
representation = []

for k in l:
    slot_found = False
    i = 0
    # firsh hash function, j = h(k)
    j = ((3*k) + 5)
    hash_value = j % 11

    while not slot_found:
        if hash_value not in keys:
            slot_found = True
            keys.append(hash_value)
            representation.append((hash_value, k))
        else:
            i += 1
            # f(i) = i * h'(k) = i * (7 - (k % 7))
            hash_value = (j + (i * (7 - (k % 7)))) % N

representation.sort(key=lambda x: x[0])
for i in range(len(keys)):
    print("{} - {}".format(representation[i][0], representation[i][1]))

0 - 13
1 - 94
2 - 23
3 - 88
4 - 39
5 - 44
6 - 11
7 - 5
8 - 12
9 - 16
10 - 20


R-10.13 What is the worst-case time for putting n entries in an initially empty hash table, with collisions resolved by chaining? What is the best case?

Worst-case: O(n^2) when all the hash values are equal and result on clustering all values of n in a single bucket of the hash table
Best case: O(n) when all the hash values are distinct and insertion to the hash table takes O(1) because there are no collisions

R-10.14 Show the result of rehashing the hash table shown in Figure 10.6 into a table of size 19 using the new hash function h(k) = 3k mod 17.

Original table:

0 - 
1 - 
2 - 54, 28, 41
3 - 
4 - 
5 - 18
6 - 
7 - 
8 - 
9 - 
10 - 10, 36
11 - 25, 38, 12, 90
12 - 

In [5]:
"""
Rehashed table
"""

N = 19
l = [54, 28, 41, 18, 10, 36, 25, 38, 12, 90]
representation = [None] * N

for k in l:
    hash_value = (3*k) % 17
    if representation[hash_value] is None:
        representation[hash_value] = [k]
    else:
        representation[hash_value].append(k)
    

for i in range(len(representation)):
    print("{} - ".format(i), end="")
    if representation[i] is not None:
        for j in range(len(representation[i])):
            print(representation[i][j], end="")
            if len(representation[i]) < j + 1:
                print(", ", end="")
    print("")

0 - 
1 - 
2 - 12
3 - 18
4 - 41
5 - 
6 - 36
7 - 25
8 - 
9 - 54
10 - 
11 - 
12 - 38
13 - 10
14 - 
15 - 90
16 - 28
17 - 
18 - 


R-10.15 Our HashMapBase class maintains a load factor λ ≤ 0.5. Reimplement that class to allow the user to specify the maximum load, and adjust the concrete subclasses accordingly.

In [ ]:
from random import randrange

from data_structures.maps.hashmap_base import HashMapBase


class LoadFactorHashMapBase(HashMapBase):
    def __init__(self, cap=11, load_factor=0.5, p=109345121):
        """Create an empty hash-table map"""
        self._table = cap * [None]
        self._load_factor = load_factor
        self._n = 0                        # number of entries in the map
        self._prime = p                    # prime for MAD compression
        self._scale = 1 + randrange(p-1)   # scale from 1 to p-1 for MAD
        self._shift = randrange(p)         # shift from 0 to p-1 for MAD
    
    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)                          # subroutine maintains self._n
        if self._n / len(self._table) > self._load_factor:     # keep load factor <= self._load_factor
            self._resize(2 * len(self._table) - 1)             # number 2^x - 1 is often prime

R-10.16 Give a pseudo-code description of an insertion into a hash table that uses quadratic probing to resolve collisions, assuming we also use the trick of replacing deleted entries with a special “deactivated entry” object.

In [ ]:
from data_structures.maps.probe_hashmap import ProbeHashMap


class QuadraticProbeHashMap(ProbeHashMap):
    def _find_slot(self, j, k):
        """Search for key k in bucket at index j.
        
        Return (success, index) tuple, described as follows:
        if match was found, succsss is True and index denotes its locations
        if no match found, success is False and index denotes first available slot
        """
        firstAvail = None
        i = 0
        while True:
            j = (j + i**2) % len(self._table)     # changed from linear to quadratic probing
            if self._is_available(j):
                if firstAvail is None:
                    firstAvail = j                # mark this as first avail
                if self._table[j] is None:
                    return (False, firstAvail)    # search has failed
            elif k == self._table[j]._key:
                return (True, j)                  # found a match
            i += 1

R-10.17 Modify our ProbeHashMap to use quadratic probing.

Look at the answer for 10.16